In [441]:
from pathlib import Path
import numpy as np
from collections import defaultdict, deque

In [476]:
data = Path('Day22.txt').read_text().splitlines()
data = sorted([[[int(coord) for coord in brick.split(',')] for brick in line.split('~') ] for i, line in enumerate(data)], key = lambda brick: brick[0][2])

# Part 1

In [477]:
max_x = max([end[0] for bricks in data for end in bricks])
max_y = max([end[1] for bricks in data for end in bricks])
max_z = max([end[2] for bricks in data for end in bricks])

print(max_z, max_y, max_x)

376 9 9


In [478]:
shaft = np.zeros((max_z + 1, max_y+1, max_x+1)) # [z, y, x] order

h, w, d = shaft.shape

for i, ([[x, y, z], [x2, y2, z2]]) in enumerate(data):
    shaft[h-z:h-z2-1:-1, y:y2+1, x:x2+1] = i+1

In [479]:
settled_shaft = np.copy(shaft)

settled = []

for i, ([x, y, z], [x2, y2, z2]) in enumerate(data):
    below = settled_shaft[h-z+1:, y:y2+1, x:x2+1]

    drops = np.nonzero(below)[0]
    
    if len(drops) > 0:
        drops = drops[0]
    else:
        drops = z-1

    if drops != 0:
        settled_shaft[h-z:h-z2-1:-1, y:y2+1, x:x2+1] = 0
        settled_shaft[h-z+drops:h-z2-1+drops:-1, y:y2+1, x:x2+1] = i+1

    settled.append([[x, y, z - drops], [x2, y2, z2-drops]])

In [480]:
removeable_bricks = set(range(1, len(data)+1))

for i, ([x, y, z], [x2, y2, z2]) in enumerate(settled):

    bricks_below = {int(brick) for brick in settled_shaft[h-z+1:h-z+2, y:y2+1, x:x2+1].ravel() if brick > 0}
    
    if len(bricks_below) == 1:
        for brick in bricks_below:
            if brick in removeable_bricks:
                removeable_bricks.remove(brick)

len(removeable_bricks)

512

# Part 2

In [481]:
supported_by = defaultdict(set)
supports = defaultdict(set)

for i, ([x, y, z], [x2, y2, z2]) in enumerate(settled):

    bricks_below = {int(brick) for brick in settled_shaft[h-z+1:h-z+2, y:y2+1, x:x2+1].ravel() if brick > 0}
    bricks_above = {int(brick) for brick in settled_shaft[h-z2-1:h-z2-2:-1, y:y2+1, x:x2+1].ravel() if brick > 0}

    supported_by[i+1] = bricks_below
    supports[i+1] = bricks_above

In [482]:
def chain_reaction(brick, supports, supported_by):
    removed_bricks = set([brick])

    to_remove = deque([b for b in supports[brick]])

    while to_remove:
        rb = to_remove.popleft()

        if supported_by[rb].intersection(removed_bricks) == supported_by[rb]:
            removed_bricks.add(rb)
            for b in supports[rb]:
                if b not in to_remove:
                    to_remove.append(b)

    return len(removed_bricks) - 1
    

In [483]:
sum([chain_reaction(brick, supports, supported_by) for brick in range(1, len(data)+1)])

98167